In [1]:
import dill
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import plotly.express as px
import plotly.graph_objs as go
print('done')

USE_PLOTLY = True


done


In [2]:
with open("experiment_config_results_gpu","rb") as current_file:
    experiment_config_results = dill.load(current_file)
with open("experiments_gpu","rb") as current_file:
    experiments = dill.load(current_file)

    
non_unique_params = np.array([experiment['parameter'] for experiment in experiments])
non_unique_config_strings = np.array([experiment['config_string'] for experiment in experiments])

par_indices = np.unique(non_unique_params,return_index=True)[1]
string_indices = np.unique(non_unique_config_strings,return_index=True)[1]

parameters = non_unique_params[np.sort(par_indices)]
config_strings = non_unique_config_strings[np.sort(string_indices)]
# config_strings[0],config_strings[1],config_strings[2] = config_strings[2],config_strings[0],config_strings[1]
config_strings[[0,1,2]] = config_strings[[2,0,1]]
print(config_strings)

def str_noneguard(obj):
    if hasattr(obj,'__name__'):
        return obj.__name__
    if obj is None:
        return ''
    if isinstance(obj,list):
        return str([str_noneguard(x) for x in obj])
    return str(obj)


print(len(config_strings))
colors=['#4285F4','#EA4335', '#FBBC04',  '#34A853','#000000','#46BDC6',      '#46BDC6',             '#F4CCCC','#C9DAF8',            'grey','purple']
print(len(colors))
print(config_strings)
print("")
print("")

print(parameters)
print("")

parameters[[3,4,5,6,7,8,9,10,11,12]] = parameters[[-4,-3,-2,5,3,4,6,8,-1,7]]
print(parameters)



['CCEu, SD=4' 'CCE, SD=4' 'JSD, SD=4' 'CCE, SD=100' 'JSD, SD=100'
 'JSD(5), SD=100' 'JSD(5)' 'MSE, SD=4' 'MSE, SD=100' 'KLD, SD=4'
 'KLD, SD=100']
11
11
['CCEu, SD=4' 'CCE, SD=4' 'JSD, SD=4' 'CCE, SD=100' 'JSD, SD=100'
 'JSD(5), SD=100' 'JSD(5)' 'MSE, SD=4' 'MSE, SD=100' 'KLD, SD=4'
 'KLD, SD=100']


['training_method' 'activation_types' 'input_layer_type' 'encoding_dim'
 'hidden_layers' 'activity_regularizer' 'sigma' 'BN_size'
 'labeled_data_percentage' 'kernel_landmarks' 'CNN_kernel_size'
 'CNN_filters' 'sampling_density']

['training_method' 'activation_types' 'input_layer_type'
 'kernel_landmarks' 'CNN_kernel_size' 'CNN_filters' 'activity_regularizer'
 'encoding_dim' 'hidden_layers' 'sigma' 'labeled_data_percentage'
 'sampling_density' 'BN_size']


In [131]:
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)

def hex_to_rgb(hex_color: str) -> tuple:
    # thanks to https://community.plotly.com/t/scatter-plot-fill-with-color-how-to-set-opacity-of-fill/29591/2
    hex_color = hex_color.lstrip("#")
    if len(hex_color) == 3:
        hex_color = hex_color * 2
    return int(hex_color[0:2], 16), int(hex_color[2:4], 16), int(hex_color[4:6], 16)

def plot_vals(i,ylim=[-175,100]):

    if USE_PLOTLY:
        fig = go.Figure()

    else:
        fig, ax = plt.subplots(figsize=[10,6])  
        ax.set_title(parameter)

    parameter = parameters[i]
    bar=0
    all_numbers = True
    
    for config_index,config_string in enumerate(config_strings):
        color=colors[config_index]
    #     print("--------- " + config_string + " -----------")
        filtered_experiments = [ex for ex in experiments if ex['parameter']==parameter and ex['config_string']==config_string]
        if len(filtered_experiments)==0:
            continue
        mappings = np.array([x['mapping'] for x in filtered_experiments])
        x_values = [x['current_var'] for x in filtered_experiments]
        for x in x_values:
            if not isinstance(x,int) and not isinstance(x,float):
                all_numbers=False

        if not all_numbers:
            for i in range(len(x_values)):
                x_values[i] = str_noneguard(x_values[i])

        results = [result for result in np.array(experiment_config_results,dtype=object)[mappings] ]
        means = np.array([np.mean(x) for x in results])
        sigmas = np.array([np.std(x) for x in results])

    #     print("mappings")
    #     display(mappings)
    #     print("")
    #     print("results")
    #     display(results)
    #     print("")
    #     print("x_values")
    #     display(x_values)
    #     print("")
    #     print("means")
    #     display(means)
    #     print("")
    #     print("sigmas")
    #     display(sigmas)
    #     print("\n\n\n")
    #     ax.plot(x_values,means)
#         print(x_values)
        if USE_PLOTLY:
            if all_numbers:
                # fig.add_trace(go.Scatter(x=x_values, y=means, error_y={"array":sigmas}, marker_color=color,name=config_string,mode='lines+markers'))
                fig.add_trace(go.Scatter(x=x_values, y=means, marker_color=color,name=config_string,mode='lines+markers'))
                fig.add_trace(go.Scatter(x=x_values, y=means+sigmas, marker_color=color,name="Upper bound", line=dict(width=0), showlegend=False, mode='lines'))
                fig.add_trace(go.Scatter(x=x_values, y=means-sigmas, marker_color=color,fillcolor=f"rgba{(*hex_to_rgb(color), 0.2)}",name="Lower bound", line=dict(width=0), showlegend=False, fill='tonexty', mode='lines'))
                
            else:
                fig.add_bar(x=x_values,y=means,error_y={"array":sigmas},marker_color=color,name=config_string)
        else:
            if all_numbers:
                ax.plot(x_values,means,'o-',label=config_string,color=color)
                ax.fill_between(x_values, (means-sigmas), (means+sigmas),alpha=.2,color=color)
            else:
                n=len(x_values)
                ind = np.arange(n)
                width = 0.1
                position = ind + (width*(1-n)/2) + bar*width
                ax.bar(position,means,width=width,yerr=sigmas,label=config_string,color=color)
                bar+=1
    
    if USE_PLOTLY:
        fig.update_yaxes(tick0=0, dtick=25)
        # fig.update_xaxes(tickangle = 0)
        fig.update_layout(yaxis_range=ylim,title=parameter, width=1000, height=600)
        fig.show()
    else:
        ax.set_ylim(ylim)
        if not all_numbers:
            print(x_values)
            print(ind)
    #         ax.axhline(0,"black")
            ax.yaxis.set_major_locator(MultipleLocator(25))    
            ax.grid(axis="y")
            
            plt.xticks(ind, x_values)
        else:
            ax.yaxis.set_major_locator(MultipleLocator(25))
            ax.grid()
        ax.axhline(0,color='black')
        ax.legend()

In [132]:
plot_vals(0)
plt.savefig("pictures/results_fig_0.svg")

<Figure size 432x288 with 0 Axes>

In [133]:
plot_vals(1)
plt.savefig("pictures/results_fig_1.svg")

<Figure size 432x288 with 0 Axes>

In [134]:
plot_vals(2)
plt.savefig("pictures/results_fig_2.svg")

<Figure size 432x288 with 0 Axes>

In [135]:
plot_vals(3)
plt.savefig("pictures/results_fig_3.svg")

<Figure size 432x288 with 0 Axes>

In [136]:
plot_vals(4)
plt.savefig("pictures/results_fig_4.svg")

<Figure size 432x288 with 0 Axes>

In [137]:
plot_vals(5)
plt.savefig("pictures/results_fig_5.svg")

<Figure size 432x288 with 0 Axes>

In [138]:
plot_vals(6)
plt.savefig("pictures/results_fig_6.svg")

<Figure size 432x288 with 0 Axes>

In [139]:
plot_vals(7)
plt.savefig("pictures/results_fig_7.svg")

<Figure size 432x288 with 0 Axes>

In [140]:
plot_vals(8)
plt.savefig("pictures/results_fig_8.svg")

<Figure size 432x288 with 0 Axes>

In [141]:
plot_vals(9)
plt.savefig("pictures/results_fig_9.svg")

<Figure size 432x288 with 0 Axes>

In [142]:
plot_vals(10)
plt.savefig("pictures/results_fig_10.svg")

<Figure size 432x288 with 0 Axes>

In [143]:
plot_vals(11)
plt.savefig("pictures/results_fig_11.svg")

<Figure size 432x288 with 0 Axes>

In [144]:
plot_vals(12,ylim=[-350,100])
plt.savefig("pictures/results_fig_12.svg")

<Figure size 432x288 with 0 Axes>

In [17]:
[str(i) + " " + str(len(experiment_config_results[x['mapping']])) for i,x in enumerate(experiments)]

['0 10',
 '1 10',
 '2 10',
 '3 10',
 '4 10',
 '5 10',
 '6 10',
 '7 10',
 '8 10',
 '9 10',
 '10 10',
 '11 10',
 '12 10',
 '13 10',
 '14 10',
 '15 10',
 '16 10',
 '17 10',
 '18 10',
 '19 10',
 '20 10',
 '21 10',
 '22 10',
 '23 10',
 '24 10',
 '25 10',
 '26 10',
 '27 10',
 '28 10',
 '29 10',
 '30 10',
 '31 10',
 '32 10',
 '33 10',
 '34 10',
 '35 10',
 '36 10',
 '37 10',
 '38 10',
 '39 10',
 '40 10',
 '41 10',
 '42 10',
 '43 10',
 '44 10',
 '45 10',
 '46 10',
 '47 10',
 '48 10',
 '49 10',
 '50 10',
 '51 10',
 '52 10',
 '53 10',
 '54 10',
 '55 10',
 '56 10',
 '57 10',
 '58 10',
 '59 10',
 '60 10',
 '61 10',
 '62 10',
 '63 10',
 '64 10',
 '65 10',
 '66 10',
 '67 10',
 '68 10',
 '69 10',
 '70 10',
 '71 10',
 '72 12',
 '73 10',
 '74 10',
 '75 10',
 '76 10',
 '77 10',
 '78 10',
 '79 10',
 '80 10',
 '81 10',
 '82 10',
 '83 10',
 '84 10',
 '85 10',
 '86 10',
 '87 10',
 '88 10',
 '89 10',
 '90 10',
 '91 10',
 '92 10',
 '93 10',
 '94 10',
 '95 10',
 '96 10',
 '97 12',
 '98 10',
 '99 10',
 '100 10',

In [18]:
# with open("experiment_config_results_merge1","rb") as current_file:
#     experiment_config_results1 = dill.load(current_file)
# with open("experiment_config_results","rb") as current_file:
#     experiment_config_results2 = dill.load(current_file)

# experiment_config_results = []
# for i in range(len(experiment_config_results1)):
#     exp1 = experiment_config_results1[i]
#     exp2 = experiment_config_results2[i]
#     experiment_config_results.append(exp1+exp2)

# display(experiment_config_results)
# with open("experiment_config_results_merge_final", "wb") as dill_file:
#     dill.dump(experiment_config_results, dill_file)

In [19]:
# with open("experiment_config_results_old", "wb") as dill_file:
#     dill.dump(experiment_config_results, dill_file)